In [1]:
import cx_Oracle
import pandas as pd
from datetime import timedelta
from datetime import datetime   
import numpy as np
import os
import re

In [2]:
# Initializing database connection
con = cx_Oracle.connect('STC_SETRAG/art@localhost/orcl.eng.engesis.com')
cur = con.cursor()

    # First Query
query=cur.execute('select * from historico_mensagens')
df = pd.DataFrame(query)
    # Second Query

cur.close()
con.close()

In [3]:
df.columns=['HM_ID_HM','HM_LOCO_MSG','HM_PRF_TRM','HM_OBC_MSG','HM_NUM_MSG','HM_TP_MSG','HM_DAT_OBC','HM_DAT_STC', 'HM_TXT_MSG','HM_TXT_TW','HM_LAT_HM','HM_LON_HM','HM_ENV_RCB','HM_MSG_VOZ','HM_NUM_FUN','HM_ID_AUX','HM_SIT_ENV','MT_ID_MT']
df = df.drop(['HM_TP_MSG','HM_TXT_TW','HM_ID_AUX','HM_OBC_MSG','HM_LAT_HM','HM_LON_HM'], axis=1)

df_sent = df.loc[df['HM_ENV_RCB']=='E']
df_received = df.loc[df['HM_ENV_RCB']=='R']

    # Removing the original dataframe
df = pd.DataFrame(None)

In [4]:
month = '2'
year = '2021'
a = timedelta(seconds=300)

In [5]:
df_sent = df_sent.loc[df_sent['HM_DAT_STC']>= year + '-' + month +'-01']
if(month == '12'):     # December
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_sent = df_sent.loc[df_sent['HM_DAT_STC']< year + '-' + str(int(month)+1) +'-01']
    # 2) Filter all messages that contain OBC time
df_sent = df_sent.loc[df_sent['HM_DAT_OBC'].notnull()]
    # 3) Filter messages to ensure that the STC time is smaller (or igual) to the OBC time (avoid negative values)
df_sent = df_sent.loc[df_sent['HM_DAT_STC']<=df_sent['HM_DAT_OBC']]
    # 4) Filter messages that contain locomotive ID
df_sent = df_sent[df_sent['HM_LOCO_MSG'].notnull()]

    # Received DataFrame
    # 1) Filter all messages that did not expired
df_received = df_received.loc[df_received['HM_SIT_ENV']=='E']
    # 2) Filter messages from within the date set by user
df_received = df_received.loc[df_received['HM_DAT_STC']>= year + '-' + month +'-01']
if(month == '12'):     # December
    df_received = df_received.loc[df_received['HM_DAT_STC']< str(int(year)+1) + '-1-01']
else:                  # Other months
    df_received = df_received.loc[df_received['HM_DAT_STC']< year + '-' + str(int(month)+1) +'-01']
    # 3) Filter all messages that contain OBC time
df_received = df_received.loc[df_received['HM_DAT_OBC'].notnull()]
    # 4) Filter all messages that have a date later than needed (avoiding future date messages)
if(month == '12'):
    df_received = df_received.loc[df_received['HM_DAT_OBC']< str(int(year)+1) +'-1-02']
else:
    df_received = df_received.loc[df_received['HM_DAT_OBC']< year + '-' + str(int(month)+1) +'-02']
    # 5) Filter messages to ensure that the OBC time is smaller (or igual) to the STC time (avoid negative values)
df_received = df_received.loc[df_received['HM_DAT_OBC']<=df_received['HM_DAT_STC']]
    # 6) Filter messages that contain locomotive ID
df_received = df_received[df_received['HM_LOCO_MSG'].notnull()]

In [6]:
totalSentMessages = len(df_sent['HM_ID_HM'])
    # Received
totalReceivedMessages = len(df_received['HM_ID_HM'])
    # Total
totalMessages = totalSentMessages + totalReceivedMessages

In [7]:
    # Sent
df_sent['DELIVERY_TIME'] = df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']
    # Converting to string in order to execute a mathematic operation
df_sent['DELIVERY_TIME'] = df_sent['DELIVERY_TIME']/np.timedelta64(1, 's')
totalSent = sum(df_sent.iloc[:,12])
averageSentDelivery = round(totalSent/totalSentMessages,0)
    # Converting from float to mm:ss
averageSentDelivery = timedelta(seconds=averageSentDelivery)
averageSentDelivery=str(averageSentDelivery)

    # Received
df_received['DELIVERY_TIME'] = df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']
    # Converting to string in order to execute a mathematic operation
df_received['DELIVERY_TIME'] = df_received['DELIVERY_TIME']/np.timedelta64(1, 's')
totalReceived = sum(df_received.iloc[:,12])
averageReceivedDelivery = round(totalReceived/totalReceivedMessages,0)


In [8]:
    # Sent
    # 3) Filter messages that took longer than 5 minutes
df_sent = df_sent.loc[(df_sent['HM_DAT_OBC']-df_sent['HM_DAT_STC']) >= a]
    # 4) Order the dataframe by HM_ID_HM
df_sent = df_sent.sort_values(by=['HM_ID_HM'])

    #Received
    # 5) Filter messages that took longer than 5 minutes
df_received = df_received.loc[(df_received['HM_DAT_STC']-df_received['HM_DAT_OBC']) >= a]
    # 6) Order the dataframe by HM_ID_HM
df_received = df_received.sort_values(by=['HM_ID_HM'])

In [9]:
# Adding new columns to count channels
df_sent['GSM'] = 0
df_sent['SAT'] = 0

df_received['GSM'] = 0
df_received['SAT'] = 0

In [10]:
#converting it from float to INT
df_sent['MT_ID_MT']=df_sent['MT_ID_MT'].astype('int')

In [11]:
# Importing files
myfile = open("D:\\kenneth\\Documents\\Clients\\SETRAG\\Tickets\\SDSTG_561_Server\\SDSTG_561\\app_STCROUTER.log")
stc_router_log = myfile.read().splitlines()
myfile.close()

myfile = open("D:\\kenneth\\Documents\\Clients\\SETRAG\\Tickets\\SDSTG_561_Server\\SDSTG_561\\app_ABR.log")
abr_log = myfile.read().splitlines()
myfile.close()

In [12]:
# Regex used to match relevant loglines 
line_regex = re.compile("Updating STC message ID: "+str(df_sent.iloc[1468,11])+" to 'E'*")

for line in stc_router_log:
    # If log line matches our regex, print to console, and output file
    if (line_regex.search(line)):
        print (line)
        file_line = line

[6880] 2021-02-12 01:28:00,306 [18] INFO  - Locomotive: 318 -> Updating STC message ID: 1857325 to 'E'


In [13]:
# Get the line number in the STC Router log
line_number = stc_router_log.index(file_line)
line_number

4682

In [14]:
# Transform the prefix from decimal to hexadecimal
prefix = hex(int(df_sent.iloc[1468,1]))
prefix = prefix[2:10]
prefix = prefix.upper()
temp = ''              #empty string delcaration for temporary variable

if len(prefix) == 2:
    prefix_hex = '-' + prefix + '-'
elif len(prefix) == 1:
    prefix_hex = '-0' + prefix + '-'

while len(prefix) > 2:     
    if len(prefix)%2 == 1:
        temp = temp + '-0' +  prefix[0] # take first character
        prefix = prefix[1:10] # take the rest
        prefix_hex =  temp + '-' + prefix + '-'
    elif len(prefix)%2 == 0:
        temp = temp + '-' + prefix[0:2] # take first two characters
        prefix = prefix[2:10] # take the rest
        prefix_hex = temp + '-' + prefix + '-'


In [15]:
prefix_hex

'-01-3E-'

In [16]:
# Find the line that contains the ACK
for i in range(0,50):
    if '-11-20-20-40-' in stc_router_log[line_number-i] and prefix_hex in stc_router_log[line_number-i]: 
        stcrouter_line = stc_router_log[line_number-i]       
        break
stcrouter_line

'[6880] 2021-02-12 01:28:00,241 [8] INFO  - Receive data[Data: 10-00-0C-02-24-01-3E-44-00-02-43-11-20-20-40-00-02-00-00-46-55-5E-51]'

In [17]:
# Slicing the string that contains the timestamp - important to avoid duplicated lines
date_hour = stcrouter_line[stcrouter_line.find(' ')+1:stcrouter_line.find(',')-2]
# Slicing the string that contains the hexadecimal
stcrouter_line = stcrouter_line[stcrouter_line.find('[Data: ')+7:] # the value '7' is so it wouldn't include he '[Data: ' part

In [19]:
# Regex used to match relevant loglines 
line_regex = re.compile(date_hour + r'.*?' + 'New packet was received' + r'.*?' + stcrouter_line)

# Loop over each log line
for line in abr_log:
# If log line matches our regex
    if (line_regex.search(line)):
        print (line)
        abr_line = line

2021-02-12 01:28:00,240 [94] DEBUG - TCP CHANNEL CONNECTION - New packet was received [Socket: 154.0.186.195:49785][Data: 10-00-0C-02-24-01-3E-44-00-02-43-11-20-20-40-00-02-00-00-46-55-5E-51]


In [20]:
abr_line

'2021-02-12 01:28:00,240 [94] DEBUG - TCP CHANNEL CONNECTION - New packet was received [Socket: 154.0.186.195:49785][Data: 10-00-0C-02-24-01-3E-44-00-02-43-11-20-20-40-00-02-00-00-46-55-5E-51]'

In [21]:
if 'TCP CHANNEL CONNECTION' in abr_line:
    df_sent.iloc[1468,13] += 1
elif 'INMARSAT CHANNEL' in abr_line: 
    df_sent.iloc[1468,14] += 1

In [22]:
df_sent.iloc[1468,]

HM_ID_HM                                                   2215170
HM_LOCO_MSG                                                    318
HM_PRF_TRM                                                    2470
HM_NUM_MSG                                                    1016
HM_DAT_OBC                                     2021-02-12 01:28:00
HM_DAT_STC                                     2021-02-12 00:44:37
HM_TXT_MSG       information de Occupations  Au tour  \n  \nAct...
HM_ENV_RCB                                                       E
HM_MSG_VOZ                                                       F
HM_NUM_FUN                                                      41
HM_SIT_ENV                                                       E
MT_ID_MT                                                   1857325
DELIVERY_TIME                                               2603.0
GSM                                                              1
SAT                                                           